In [ ]:
#pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu130
#pip install langchain langchain_community langchainhub chromadb sentence-transformers langchain-openai
#pip install -U numpy faiss-cpu FlagEmbedding openai
#pip install langchain-experimental langchain-core
#%pip install -U langchain-ollama

windows + pip + cuda 11.8 %pip install torch torchvision torchaudio --index-url https://download.python.org/whl/cu118

windows + pip + cpu %pip install torch torchvision torchaudio

linux + pip + cuda 12.1 %pip install torch torchvision torchaudio --index-url https://download.python.org/whl/cu118

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.9.0+cu130
True


In [3]:
#git clone https://github.com/FlagOpen/FlagEmbedding.git
#cd FlagEmbedding
#pip install -e .

In [1]:
# Import libaries
from langchain_community.document_loaders import JSONLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
import numpy as np
import json
import os

In [2]:
from FlagEmbedding import BGEM3FlagModel
# If not work FlagEmbedding, you need to install "pip install git+https://github.com/FlagOpen/FlagEmbedding.git"

In [3]:
with open("../resources/cpe.json", "r", encoding="utf-8") as f:
    data = json.load(f)

docs = []

In [4]:
for item in data:
    tipo = item.get("tipo", "").capitalize()

    if tipo == "Introduccion":
        text = f"Título: {item.get('titulo', '')}\n" \
                f"Subtítulo: {item.get('subtitulo', '')}\n\n" \
                f"{item.get('contenido', '')}"

    elif tipo == "Articulo":
        text = (
            f"Parte {item.get('parte_num', '')}: {item.get('parte_nom', '')}\n"
            f"Título {item.get('titulo_num', '')}: {item.get('titulo_nom', '')}\n"
            f"Capítulo {item.get('capitulo_num', '')}: {item.get('capitulo_nom', '')}\n"
            f"Sección {item.get('seccion_num', '')}: {item.get('seccion_nom', '')}\n"
            f"Artículo {item.get('art_num', '')}: {item.get('nombre_juridico', '')}\n\n"
            f"{item.get('contenido', '')}"
        )
        
    elif tipo == "Disposición":
        text = (
            f"Disposición {item.get('disposicion', '')}\n"
            f"Nombre jurídico: {item.get('nombre_juridico', '')}\n\n"
            f"{item.get('contenido', '')}"
        )

    # Pongo este else, porque sino me sale error en page_content=text, porque es posible que text este vacio
    else:
        text = item.get("contenido", "")

    docs.append(Document(page_content=text, metadata={"tipo": tipo}))

print(f"Se cargaron {len(docs)} documentos desde el JSON.")
#print(docs[426])


Se cargaron 426 documentos desde el JSON.


In [5]:
# test
print(docs[199])

page_content='Parte Segunda: Estructura y organización funcional del estado
Título 3: Órgano Judicial y Tribunal Constitucional Plurinacional
Capítulo Sexto: Tribunal Constitucional Plurinacional
Sección None: None
Artículo 198: Forma de Elección de los Magistrados del Tribunal Constitucional Plurinacional

Las Magistradas y los Magistrados del Tribunal Constitucional Plurinacional se elegirán mediante sufragio universal, según el procedimiento, mecanismo y formalidades de los miembros del Tribunal Supremo de Justicia.' metadata={'tipo': 'Articulo'}


In [8]:
# Embedding Model
model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True) 

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

LookupError: <ContextVar name='shell_parent' at 0x000001FFD7F1AFC0>

In [10]:
texts = [doc.page_content for doc in docs if doc.page_content.strip() != ""]

embeddings = model.encode(texts, return_dense=True, return_sparse=True, return_colbert_vecs=True)

pre tokenize: 100%|██████████| 2/2 [00:00<00:00, 41.41it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 2/2 [00:48<00:00, 24.32s/it]


In [11]:
embeddings.values()

dict_values([array([[ 0.0412  ,  0.0313  , -0.011986, ...,  0.01423 , -0.00476 ,
        -0.02759 ],
       [ 0.03732 ,  0.00144 , -0.0488  , ...,  0.00863 ,  0.00226 ,
        -0.008255],
       [ 0.05194 , -0.001332, -0.01772 , ...,  0.004192,  0.00993 ,
        -0.03488 ],
       ...,
       [-0.02994 ,  0.000939,  0.004303, ..., -0.04404 , -0.018   ,
        -0.002092],
       [ 0.03598 ,  0.03546 ,  0.004204, ...,  0.01802 , -0.02567 ,
        -0.03522 ],
       [ 0.04404 ,  0.0259  , -0.011215, ...,  0.01255 ,  0.001907,
        -0.01915 ]], shape=(426, 1024), dtype=float16), [defaultdict(<class 'int'>, {'211025': np.float16(0.1272), '12': np.float16(0.0163), '43924': np.float16(0.1602), '37534': np.float16(0.2307), '14962': np.float16(0.132), '51825': np.float16(0.2065), '90': np.float16(0.04184), '8273': np.float16(0.1057), '5233': np.float16(0.03613), '438': np.float16(0.08527), '86': np.float16(0.03778), '239': np.float16(0.1292), '155972': np.float16(0.3506), '31256': np.flo

In [12]:
np.save("cpe_e.npy", embeddings)
print(f"{len(texts)} artículos procesados.")


426 artículos procesados.


In [13]:
#pip install chromadb

In [16]:
import chromadb

# Persistent in disk
client = chromadb.PersistentClient(path="./vectordb")

# In memory (RAM - only for testing)
#client = chromadb.Client()

# In case, we hacve a chroma server 
#client = chromadb.HttpClient(host="localhost", port=8000, ssl=False) #Change port and ssl value if you need



In [16]:
emb = np.load("cpe_e.npy", allow_pickle=True).item()
print(emb.keys())

dict_keys(['dense_vecs', 'lexical_weights', 'colbert_vecs'])


In [17]:
dense = emb["dense_vecs"]
dense_list = dense.tolist()

sparse = emb["lexical_weights"]
colbert = emb["colbert_vecs"]

In [18]:
collection = client.get_or_create_collection(
    name="constitucion",
    metadata={"Modelo de embedding": "bge-m3", "Tipo": "Sparse para Chroma"}
)

In [ ]:
# Por la forma en la que esta organizada la constitucion, los ids y metadatos tienen esta estructura
ids = []
metas = []
cont = 0

for i in range(len(texts)):
    if (i<2):
        ids.append(f"introduccion_{i+1}")
        metas.append({"tipo": "introduccion", "número": i+1})
    elif(i<399):
        ids.append(f"artículo_{i-1}")
        metas.append({"tipo": "artículo", "número": i-1})
    elif(i==399):
        ids.append(f"disposición_398_A")
        metas.append({"tipo": "disposicion", "número": "398_A"})
    elif(i==400):
        ids.append(f"disposición_398_B")
        metas.append({"tipo": "disposicion", "número": "398_B"})
    elif(i<414):
        ids.append(f"artículo_{i-2}")
        metas.append({"tipo": "artículo", "número": i-2})
    else:
        cont += 1
        ids.append(f"disposición_{cont}")
        metas.append({"tipo": "disposicion", "número": cont})


In [20]:
collection.add(
    ids=ids,
    documents=texts,
    embeddings=dense_list,
    metadatas=metas
)

In [21]:
# In case, you want to add a new embedding:
'''
collection.add(
    ids=ids, (Los ids deben ser str, no reconoce otros tipos de datos y si pones algun id existen, la db lo cambia a uno random o sale error)
    documents=texts, (opcional)
    embeddings=dense_list,
    metadatas=metas (opcional)
)
'''

'\ncollection.add(\n    ids=ids, (Los ids deben ser str, no reconoce otros tipos de datos y si pones algun id existen, la db lo cambia a uno random o sale error)\n    documents=texts, (opcional)\n    embeddings=dense_list,\n    metadatas=metas (opcional)\n)\n'

In [22]:
print(client.list_collections())
collection = client.get_collection("constitucion")
print("Total documentos:", collection.count())

[Collection(name=constitucion)]
Total documentos: 426


In [ ]:
#test
query = "Quiero saber acerca del articulo 400"
query_emb = model.encode(query)["dense_vecs"]

resultados = collection.query(
    query_embeddings=[query_emb],
    n_results=4
)

for i in range(len(resultados["documents"][0])):
    print(resultados["ids"][0][i])
    print(resultados["documents"][0][i], "\n")

artículo_400
Parte Cuarta: Estructura y organización económica del Estado
Título 2: Medio Ambiente, Recursos Naturales, Tierra y Territorio
Capítulo Noveno: Tierra y territorio
Sección None: None
Artículo 400: Prohibición de la Fragmentación de la Pequeña Propiedad

Por afectar a su aprovechamiento sustentable y por ser contrario al interés colectivo, se prohíbe la división de las propiedades en superficies menores a la superficie máxima de la pequeña propiedad reconocida por la ley que, para su establecimiento, tendrá en cuenta las características de las zonas geográficas. El Estado establecerá mecanismos legales para evitar el fraccionamiento de la pequeña propiedad. 

artículo_399
Parte Cuarta: Estructura y organización económica del Estado
Título 2: Medio Ambiente, Recursos Naturales, Tierra y Territorio
Capítulo Noveno: Tierra y territorio
Sección None: None
Artículo 399: Nuevos Límites de la Propiedad Agraria Zonificada y Superficies Excedentes

I. Los nuevos límites de la propie

In [35]:
#pip install hf_xet